In [1]:
import tensorflow as tf
tf.enable_eager_execution()

/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fmsnew/miniconda3/envs

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

import json
import functools

In [3]:
import os
os.environ['AUTOGRAPH_VERBOSITY'] = '10'

In [4]:
config = _config.build_config()
config['use_tpu'] = False
config['train_data_files'] = ['train_1.tfrecords', 'train_2.tfrecords', 'train_3.tfrecords', 'train_4.tfrecords']
config['valid_data_file'] = ['validation.tfrecords']
config['test_data_file'] = ['test.tfrecords']
config['sample_data_file'] = ['sample.tfrecords']
config['train_epochs'] = 36

matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
      [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
      [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
      [0, 0, 0, 0, 0, 0, 0]]   # output layer
# Operations at the vertices of the module, matches order of matrix
labels=['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3', 'output']


matrix = np.array(matrix)
labels = np.array(labels)

spec = model_spec.ModelSpec(matrix, labels)
model_dir = '../my_model_dir'

In [5]:
run_config = tf.contrib.tpu.RunConfig(model_dir=model_dir)

# check model accuracy

In [6]:
input_test = cifar.CIFARInput('test', config)

In [7]:
est = tf.contrib.tpu.TPUEstimator(
      use_tpu=config['use_tpu'],
      model_fn=model_builder.build_model_fn(spec, config, None),
      config=run_config,
      params={'use_KD':False},
      train_batch_size=config['batch_size'],
      eval_batch_size=config['batch_size'],
      predict_batch_size=1000)

INFO:tensorflow:Using config: {'_model_dir': '../my_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2e035eb7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, inpu

In [8]:
est_preds = est.predict(input_fn=input_test.input_fn, yield_single_examples=False)

In [9]:
all_pred_logits = []
for preds in est_preds:
    all_pred_logits.append(preds['logits'])
all_pred_logits = np.vstack(all_pred_logits)

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 32, 32, 32, 32, 32, 128]
INFO:tensorflow:vertex_channels: [128, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 64, 64, 64, 64, 64, 256]
INFO:tensorflow:vertex_channels: [256, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../my_model_dir/model.ckpt-5625
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop

In [10]:
preds = np.argmax(all_pred_logits, axis=1)

In [11]:
preds

array([3, 8, 8, ..., 5, 1, 7])

In [12]:
dataset = input_test.input_fn({'batch_size':1000, 'use_KD':False})

In [13]:
answers = []
cnt = 0
for records in dataset:
    answers.append(records[1].numpy())
answers = np.hstack(answers)

In [14]:
answers.shape

(10000,)

In [15]:
print('test accuracy:', np.mean(answers == preds))

test accuracy: 0.9136


# Make KD dataset

In [6]:
all_files = config['train_data_files'] + config['valid_data_file'] + config['sample_data_file'] + config['test_data_file']

In [7]:
WIDTH = 32
HEIGHT = 32
RGB_MEAN = [125.31, 122.95, 113.87]
RGB_STD = [62.99, 62.09, 66.70]

In [8]:
def _preprocess(image):
  """Perform standard CIFAR preprocessing.

  Pads the image then performs a random crop.
  Then, image is flipped horizontally randomly.

  Args:
    image: image Tensor with shape [height, width, 3]

  Returns:
    preprocessed image with the same dimensions.
  """
  # Pad 4 pixels on all sides with 0
  image = tf.image.resize_image_with_crop_or_pad(
      image, HEIGHT + 8, WIDTH + 8)

  # Random crop
  image = tf.random_crop(image, [HEIGHT, WIDTH, 3], seed=0)

  # Random flip
  image = tf.image.random_flip_left_right(image, seed=0)

  return image


def _parser(use_preprocessing, serialized_example):
  """Parses a single tf.Example into image and label tensors."""
  features = tf.parse_single_example(
      serialized_example,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
      })
  image = tf.decode_raw(features['image'], tf.uint8)
  image.set_shape([3 * HEIGHT * WIDTH])
  image = tf.reshape(image, [3, HEIGHT, WIDTH])
  # TODO(chrisying): handle NCHW format
  image = tf.transpose(image, [1, 2, 0])
  image = tf.cast(image, tf.float32)
  if use_preprocessing:
    image = _preprocess(image)
  image -= tf.constant(RGB_MEAN, shape=[1, 1, 3])
  image /= tf.constant(RGB_STD, shape=[1, 1, 3])
  label = tf.cast(features['label'], tf.int32)
  return image, label

In [9]:
def _set_batch_dimension(batch_size, images, labels):
    images.set_shape(images.get_shape().merge_with(
      tf.TensorShape([batch_size, None, None, None])))
    labels.set_shape(labels.get_shape().merge_with(
      tf.TensorShape([batch_size])))

    return images, labels


In [10]:
def _dummy_imput_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(params['file'])
    dataset = dataset.prefetch(buffer_size=batch_size)

   
   # Parse, preprocess, and batch images
    parser_fn = functools.partial(_parser, False)
    dataset = dataset.apply(
        tf.contrib.data.map_and_batch(
            parser_fn,
            batch_size=batch_size,
            num_parallel_batches=None,
            drop_remainder=True))

    # Assign static batch size dimension
    dataset = dataset.map(functools.partial(_set_batch_dimension, batch_size))

    # Prefetch to overlap in-feed with training
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    

    return dataset

In [12]:
for file in all_files:
#for file in ['sample.tfrecords']:
    raw_dataset = tf.data.TFRecordDataset([file])
    params = {'file':file, 'use_KD':False}
    est = tf.contrib.tpu.TPUEstimator(
          use_tpu=False,
          model_fn=model_builder.build_model_fn(spec, config, None),
          config=run_config,
          params=params,
          train_batch_size=config['batch_size'],
          eval_batch_size=config['batch_size'],
          predict_batch_size=100)


    est_preds = est.predict(input_fn=_dummy_imput_fn, yield_single_examples=False)

    all_pred_logits_aug = []
    for preds in est_preds:
        all_pred_logits_aug.append(preds['logits'])
    all_pred_logits_aug = np.vstack(all_pred_logits_aug)

    out_file = file.replace('.', '_KD.')

    with tf.python_io.TFRecordWriter(out_file) as record_writer:
        i = 0
        for raw_record in raw_dataset:
            example = tf.train.Example()
            example.ParseFromString(raw_record.numpy())
            img = example.features.feature['image']
            label = example.features.feature['label']
            preds = all_pred_logits_aug[i]
            new_label = np.hstack((label.int64_list.value[0], preds))
            feat_preds = tf.train.Feature(float_list=tf.train.FloatList(value=new_label))
            example = tf.train.Example(features=tf.train.Features(
                      feature={
                          'image': img,
                          'label': feat_preds
                      }))
            record_writer.write(example.SerializeToString())
            i += 1
            break

INFO:tensorflow:Using config: {'_model_dir': '../my_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f971b655898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, inpu

INFO:tensorflow:vertex_channels: [512, 128, 128, 128, 128, 128, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../my_model_dir/model.ckpt-5625
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:Using config: {'_model_dir': '../my_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.se

In [17]:
all_pred_logits_aug

[]

In [61]:
se = example.SerializeToString()

In [62]:
tse = tf.convert_to_tensor(se)

In [63]:
tse

<tf.Tensor: id=151873, shape=(), dtype=string, numpy=b'\n\xce\x18\n\x90\x18\n\x05image\x12\x86\x18\n\x83\x18\n\x80\x18\xe5\xec\xea\xe7\xec\xf2\xf6\xf6\xf2\xec\xe8\xe8\xe9\xe8\xe3\xdc\xd6\xd2\xd0\xd0\xd2\xd4\xd4\xd5\xd7\xd8\xdc\xd9\xd6\xd9\xdd\xde\xde\xef\xe9\xe8\xea\xf0\xf4\xf4\xf0\xe9\xe6\xe5\xe6\xe4\xde\xd8\xd4\xd2\xd1\xd2\xd3\xd5\xd7\xd8\xd9\xd6\xdd\xdc\xdc\xdf\xe3\xd2\xd5\xea\xe7\xe6\xe7\xe8\xea\xea\xe6\xe2\xe1\xe2\xdf\xdc\xd6\xd0\xcc\xcd\xcf\xd0\xd2\xd6\xd3\xd3\xd5\xd3\xda\xd1\xd4\xdc\xdc\xca\xa0\xb0\xbc\xc5\xd1\xdd\xe4\xe6\xe6\xe6\xe6\xe5\xe1\xdc\xd7\xd0\xcd\xd2\xd3\xd5\xd4\xc5\xbb\xca\xd6\xd1\xdd\xc0\xc7\xd8\xd3\xd0\x88\x89\x92\x8c\x81\x8c\x99\xaa\xba\xc1\xd9\xe6\xe4\xe3\xd7\xce\xd8\xde\xdb\xda\xd3\xb1\xa5\xbf\xd7\xd0\xd9\xbc\xc4\xcd\xc9\xcci\xaa\xce\xa6~\x84\x82il\x80\x8b\x9f\xae\xbd\xb3\xbb\xd9\xdf\xd7\xd3\xca\xaa\x92\xb2\xc2\xbc\xbd\xa0\xa8\xad\xae\xbc^\xa9\xbe\xaaLv\x9dw~\x8dx\x91\x93\x90\x88\x82\x92\xb7\xb4\xad\xb4\x90_\xaa\xa2\x9a\xaa\x8e\x98\xa4\xa6\xae\x86\xbb\xb1\xa6Dq\

In [60]:
features = tf.parse_single_example(
      tse,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([11], tf.float32),
      })


InvalidArgumentError: Key: label.  Can't parse serialized Example. [Op:ParseSingleExample]

In [55]:
features

{'image': <tf.Tensor: id=151864, shape=(), dtype=string, numpy=b'\xe5\xec\xea\xe7\xec\xf2\xf6\xf6\xf2\xec\xe8\xe8\xe9\xe8\xe3\xdc\xd6\xd2\xd0\xd0\xd2\xd4\xd4\xd5\xd7\xd8\xdc\xd9\xd6\xd9\xdd\xde\xde\xef\xe9\xe8\xea\xf0\xf4\xf4\xf0\xe9\xe6\xe5\xe6\xe4\xde\xd8\xd4\xd2\xd1\xd2\xd3\xd5\xd7\xd8\xd9\xd6\xdd\xdc\xdc\xdf\xe3\xd2\xd5\xea\xe7\xe6\xe7\xe8\xea\xea\xe6\xe2\xe1\xe2\xdf\xdc\xd6\xd0\xcc\xcd\xcf\xd0\xd2\xd6\xd3\xd3\xd5\xd3\xda\xd1\xd4\xdc\xdc\xca\xa0\xb0\xbc\xc5\xd1\xdd\xe4\xe6\xe6\xe6\xe6\xe5\xe1\xdc\xd7\xd0\xcd\xd2\xd3\xd5\xd4\xc5\xbb\xca\xd6\xd1\xdd\xc0\xc7\xd8\xd3\xd0\x88\x89\x92\x8c\x81\x8c\x99\xaa\xba\xc1\xd9\xe6\xe4\xe3\xd7\xce\xd8\xde\xdb\xda\xd3\xb1\xa5\xbf\xd7\xd0\xd9\xbc\xc4\xcd\xc9\xcci\xaa\xce\xa6~\x84\x82il\x80\x8b\x9f\xae\xbd\xb3\xbb\xd9\xdf\xd7\xd3\xca\xaa\x92\xb2\xc2\xbc\xbd\xa0\xa8\xad\xae\xbc^\xa9\xbe\xaaLv\x9dw~\x8dx\x91\x93\x90\x88\x82\x92\xb7\xb4\xad\xb4\x90_\xaa\xa2\x9a\xaa\x8e\x98\xa4\xa6\xae\x86\xbb\xb1\xa6Dq\x9f\x9b\x9eqU\x94\xa1\x92\xa1\x81s\x8e\xab\xb1\xbd\x8